In [2]:
from ipynb.fs.full.utility import *

In [3]:
%matplotlib inline
import numpy as np, scipy, matplotlib.pyplot as plt, IPython.display as ipd
import librosa, librosa.display
from sklearn.preprocessing import normalize
from sklearn.decomposition import PCA
from scipy import stats
#import stanford_mir; stanford_mir.init()

import torch
import torch.nn as nn
import torchvision

In [4]:
def combineLabel(arr, lab):
    label = []
    for index in range(arr.shape[0]):
        time = (index+0.5) * 512 /11025
        for item in lab:
            if time > item[1]:
                continue
            else:
                label.append(item[2])
                break
    return arr[:len(label)], np.array(label)

In [5]:
import numpy as np
def NormalizeData(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))

In [6]:
import os
def root(x):
    return os.path.splitext(os.path.basename(x))[0]

In [6]:
data = None
data_len = []
# latest method of preprocessing
def create_npzs():
    for i, (audio, anno) in enumerate(zip(audios, annos)):
       # try:
        # load song
        x, sr = librosa.load(audio, sr=11025)
        fmin = librosa.midi_to_hz(45)

        # get cqt, lab
        C = librosa.cqt(x, sr=sr, fmin=fmin, n_bins=180, bins_per_octave=36)
        if (i == 0):
            data = C
        else:
            data = np.append(data, C, axis=1)
        data_len.append(C.shape[0])
        print(str(i) + ': ' + str(C.shape[1]))
    
    pca = PCA(.99)
    prinComp = pca.fit_transform(np.abs(data).T)
    norm = stats.zscore(prinComp)
    
    for i, anno in enumerate(annos):
        lab = loadLab(anno)
        
        cqt = data[:data_len[i]]

        # combine the label
        cqt, label = combineLabel(norm, lab)

        # get the chord idx
        V = np.vectorize(commonChordToIdx)
        chordidx = V(label)

        # save as npz
        path = '/userhome/cs2/u3556571/8th/npzs/' + root(anno)
        np.savez_compressed(path, cqt=cqt, label=chordidx)

        print('Finish ' + str(i))
        
        data = np.delete(data, range(data_len[i]), axis=1)
        '''except Exception as e:
            print('Skip ' + str(i) + str(e))
            continue'''

In [ ]:
create_npzs()

0: 3786


In [8]:
def throw_border_sample(rawcqt, lab):
    nbin = rawcqt.shape[0]
    time = float(lab[len(lab)-1][1])
    binTime = time/nbin
    cqt = []
    label = []
    for binIdx in range(nbin):
        lowerTime = binTime*(binIdx)
        upperTime = binTime*(binIdx+1)
        for slot in lab:
            if lowerTime >= slot[1]:
                continue
            elif upperTime > slot[1]:
                break
            else:
                cqt.append(rawcqt[binIdx])
                label.append(slot[2])
                break
    cqt = np.array(cqt)
    label = np.array(label)
    return cqt, label

In [ ]:
# old method of preprocessing
for i, (audio, anno) in enumerate(zip(audios, annos)):
    try:
        # load song
        x, sr = librosa.load(audio)
        fmin = librosa.midi_to_hz(36)
        hop_length = 256
        
        # get cqt
        C = librosa.cqt(x[:], sr=sr, fmin=fmin, n_bins=72, hop_length=hop_length)
        logC = librosa.amplitude_to_db(np.abs(C))
        trans = logC.transpose()
        lab = loadLab(anno)
        
        # filter the bin
        cqt, label = throw_border_sample(trans, lab)
        
        # normalize the cqt
        normalized = NormalizeData(cqt)
        
        # get the chord idx
        chordToIdxV = np.vectorize(majminChordToIdx)
        chordidx = chordToIdxV(label)

        # save as npz
        path = '/userhome/cs2/u3556571/5th_majmin/npzs/' + root(anno)
        np.savez_compressed(path, cqt=normalized, label=chordidx)
        
        print('Finish ' + str(i))
        break
    except Exception as e:
        print('Skip ' + str(i) + str(e))
        continue